## 2. Agent

<div style="text-align: right"> Initial issue : 2025.11.05 </div>
<div style="text-align: right"> last update : 2025.11.05 </div>

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### 에이전트 작동방식
- 목표 달성을 위해 도구를 반복적으로 실행  
- 즉, 중단 조건이 충족 될때까지 실행되며, 모델이 최종 출력을 생성하거나 반복 제한에 도달하면 중단

```
입력 -> 모델 -> 도구 호출 -> 도구 실행 -> 모델 -> ... -> 최종 출력
```

### 모델
- 모델은 에이전트의 추론엔진
- 모델을 정적 또는 동적으로 지정가능

정적모델: 에이전트 생성시 한번 구성되며 실행 중 변경되지 않음

In [2]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

In [3]:
# 방법 1: 모델 식별자 문자열을 사용한 간단한 방법
agent = create_agent("openai:gpt-4.1-mini", tools=[])

# 방법 2: 모델 인스턴스를 직접 초기화하여 더 세밀한 제어
model = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature=0.1,
    max_completion_tokens=1000,
    timeout=30 # 초
)

agent2 = create_agent(model=model, tools=[])

동적 모델: 런타임에 현재 상태와 컨텍스트를 기반으로 선택
- 정교한 라우팅 로직과 비용 최적화 구현이 가능함

In [4]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

In [5]:
# 기본 모델과 고급 모델 정의
basic_model = ChatOpenAI(model="gpt-4.1-mini")
advanced_model = ChatOpenAI(model="gpt-4.1")

In [6]:
@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler)-> ModelResponse:
    """대화 복잡도에 따라 모델 선택"""
    print(request)
    message_count = len(request.state["messages"])

    # 긴 대화에는 고급 모델 사용
    if message_count > 10:
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)

In [7]:
agent = create_agent(
    model = basic_model,
    tools=[],
    middleware=[dynamic_model_selection]
)

In [8]:
from langchain_teddynote.messages import stream_graph
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig

stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해줘")]
    },
    config=RunnableConfig(),
)

ModelRequest(model=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x110b9b3e0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x110b9ba70>, root_client=<openai.OpenAI object at 0x110b991f0>, root_async_client=<openai.AsyncOpenAI object at 0x110b994c0>, model_name='gpt-4.1-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True), system_prompt=None, messages=[HumanMessage(content='머신러닝의 동작 원리에 대해서 설명해줘', additional_kwargs={}, response_metadata={}, id='78991d01-0e26-4c65-8116-3b5ecc31ae0b')], tool_choice=None, tools=[], response_format=None, state={'messages': [HumanMessage(content='머신러닝의 동작 원리에 대해서 설명해줘', additional_kwargs={}, response_metadata={}, id='78991d01-0e26-4c65-8116-3b5ecc31ae0b')]}, runtime=Runtime(context=None, store=None, stream_writer=<function Pregel.stream.<locals>.stream_writer at 0x110246de0>, previous=None), model_settings={})

🔄 Node: model 🔄
- - - - - - - - - -

### 도구
- 에이전트에게 행동(action)을 취할 수 있는 능력 부여
- 에이전트는 다음을 지원
    - 순차적으로 여러도구 호출
    - 적절한 경우 병렬 도구 호출
    - 이전 결과를 기반으로 동적 도구 선택
    - 도구 재시도 로직 및 오류 처리
    - 도구 호출간 상태 지속성

도구 정의

In [9]:
from langchain.tools import tool
from langchain.agents import create_agent


@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"


@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    # 일부러 오류 발생
    raise Exception("날씨 정보를 가져오는 중 오류가 발생하였습니다.")


# 도구 리스트를 에이전트에 전달
agent = create_agent("openai:gpt-4.1-mini", tools=[search, get_weather])

In [10]:
stream_graph(agent, inputs={"messages": [HumanMessage(content="서울 날씨 조회해줘")]})


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 


Exception: 날씨 정보를 가져오는 중 오류가 발생하였습니다.

도구 오류 처리: 오류 처리를 커스터마이징하려면 `@wrap_tool_call` 데코레이터를 사용하여 미들웨어를 생성

In [11]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage


@wrap_tool_call
def handle_tool_errors(request, handler):
    """도구 실행 오류를 커스텀 메시지로 처리"""
    try:
        return handler(request)
    except Exception as e:
        # 모델에 커스텀 오류 메시지 반환
        return ToolMessage(
            content=f"[에러] 도구 호출시 오류가 발생했어요. 다시 시도해주세요. ({str(e)})",
            tool_call_id=request.tool_call["id"],
        )


agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search, get_weather],
    middleware=[handle_tool_errors],
)

In [12]:
stream_graph(agent, inputs={"messages": [HumanMessage(content="서울 날씨 조회해줘")]})


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[에러] 도구 호출시 오류가 발생했어요. 다시 시도해주세요. (날씨 정보를 가져오는 중 오류가 발생하였습니다.)
🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
현재 날씨 정보를 가져오는 중에 오류가 발생했습니다. 잠시 후 다시 시도해 주시겠어요?

### 프롬프트

In [13]:
agent = create_agent(
    "openai:gpt-4.1-mini",
    tools=[search, get_weather],
    system_prompt="You are a helpful assistant. Be concise and accurate.",
)

동적 시스템 프롬프트: 런타임 컨텍스트나 에이전트 상태를 기반으로 시스템 프롬프트를 수정해야 하는 경우 미들웨어를 사용 가능

- 핵심: `request.runtime.context`

In [14]:
from typing import TypedDict
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


class Context(TypedDict):
    answer_type: str


@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """사용자 역할에 따라 시스템 프롬프트 생성"""
    answer_type = request.runtime.context.get("answer_type", "default")
    base_prompt = "You are a helpful assistant."

    if answer_type == "default":
        return f"{base_prompt} Answer in Korean. 간결하게 답변해줘."
    elif answer_type == "sns":
        return f"{base_prompt} Answer in Korean. SNS 형식으로 답변해줘."
    elif answer_type == "article":
        return f"{base_prompt} Answer in Korean. 뉴스 기사 형식으로 답변해줘."

    return base_prompt


agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search],
    middleware=[user_role_prompt],
    context_schema=Context,
)

In [15]:
# 컨텍스트에 따라 시스템 프롬프트가 동적으로 설정됩니다
stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해줘")]
    },
    context=Context(answer_type="sns"),
)


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
📢 머신러닝 동작 원리 간단 정리!

1️⃣ 데이터 수집: 학습할 데이터를 모아요  
2️⃣ 전처리: 데이터 정리 및 가공  
3️⃣ 모델 선택: 문제에 맞는 알고리즘 선택  
4️⃣ 학습: 모델이 데이터를 보고 패턴을 배워요  
5️⃣ 평가: 성능 테스트하여 얼마나 잘 배웠는지 확인  
6️⃣ 예측/적용: 새로운 데이터에 모델을 적용해 결과 도출!

쉽게 말하면, 과거 데이터를 통해 스스로 학습하고 미래를 예측하는 기술! 💻✨

#머신러닝 #인공지능 #데이터과학

### 에이전트 호출
에이전트 상태에 업데이트를 전달하여 호출 가능  
모든 에이전트는 상태에 메시지 시퀀스를 포함??

In [16]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's the weather in San Fransisco?"}]},
    context={"user_role": "expert"}
)

print(result)

{'messages': [HumanMessage(content="What's the weather in San Fransisco?", additional_kwargs={}, response_metadata={}, id='a1aa001d-9b4a-4402-9a18-c0f7847e74fb'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 66, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CYoFAUrBZ9YROwiSUkrawwNqdrpoc', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--40d38361-4282-4bb3-bbd1-b103711471a5-0', tool_calls=[{'name': 'search', 'args': {'query': 'San Francisco weather'}, 'id': 'call_5dADlLdv6mowQgmk5WAbuzcA', 'type': 'tool_call'}], usage_metadata={'input_tokens': 66, 'output_toke

### 고급 개념

### 구조화된 출력  
- 특정 형식으로 에이전트 출력을 반환하고 싶을때, response_format 매개변수를 통해 구조화된 출력 제공

ToolStrategy: 인공적인 도구 호출을 통해 구조화된 출력 생성

In [17]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

# 응답 스키마 정의
class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str


agent = create_agent(
    model="openai:gpt-4.1-mini", tools=[], response_format=ToolStrategy(ContactInfo)
)


In [18]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567",
            }
        ]
    }
)

print(result["structured_response"])
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

name='John Doe' email='john@example.com' phone='(555) 123-4567'


ProviderStrategy: 모델 제공자의 네이티브 구조화된 출력 사용. 더 안정적이지만 구조화된 출력을 지원하는 모델만 작동(예: openai)

In [19]:
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    model="openai:gpt-4.1", response_format=ProviderStrategy(ContactInfo)
)

In [20]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567",
            }
        ]
    }
)

print(result["structured_response"])

name='John Doe' email='john@example.com' phone='(555) 123-4567'


### 메모리
커스텀 상태 스키마를 사용하여 메모리 관리 가능

### 미들웨어를 통항 상태 정의
커스텀 상태가 특정 미들웨어 훅 및 해당 미들웨어에 연결된 도구에서 액세스해야 하는 경우 미들웨어를 사용하여 커스텀 상태를 정의

In [21]:
from typing import Any
from langchain.agents import AgentState
from langchain.agents.middleware import AgentMiddleware


# 커스텀 상태 스키마 정의
class CustomState(AgentState):
    user_preferences: dict


class CustomMiddleware(AgentMiddleware):
    state_schema = CustomState
    tools = []

    def before_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
        # 모델 호출 전 커스텀 로직
        pass

In [22]:
agent = create_agent("openai:gpt-4.1-mini", tools=[], middleware=[CustomMiddleware()])

# 에이전트는 이제 메시지 외에 추가 상태를 추적할 수 있습니다
result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "I prefer technical explanations"}],
        "user_preferences": {"style": "technical", "verbosity": "detailed"},
    }
)

### State_schema를 통한 상태 정의

In [23]:
from langchain.agents import AgentState


class CustomState(AgentState):
    user_preferences: dict


agent = create_agent("openai:gpt-4.1-mini", tools=[], state_schema=CustomState)

In [24]:
agent = create_agent("openai:gpt-4.1-mini", tools=[], state_schema=CustomState)

result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "I prefer technical explanations"}],
        "user_preferences": {"style": "technical", "verbosity": "detailed"},
    }
)

### 스트리밍

In [25]:
for chunk in agent.stream(
    {
        "messages": [
            {"role": "user", "content": "Search for AI news and summarize the findings"}
        ]
    },
    stream_mode="values",
):
    # 각 청크에는 해당 시점의 전체 상태가 포함됩니다
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif hasattr(latest_message, "tool_calls") and latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

Agent: Search for AI news and summarize the findings
Agent: I don't have real-time internet access to search for the latest AI news. However, I can provide a summary of recent trends and developments in AI based on information available up to mid-2024. If you are looking for the very latest updates, I recommend checking trusted news sources like AI-focused blogs, technology news websites, or official announcements from leading AI companies.

Here’s a summary of key recent trends and developments in AI:

1. **Generative AI Advancements**: Large language models (LLMs) and generative AI tools continue to improve, with companies releasing more powerful and efficient versions. These models are being integrated into various applications, including coding assistants, content creation, and customer service.

2. **AI in Healthcare**: AI is increasingly used for diagnostics, drug discovery, personalized medicine, and managing healthcare data. New studies show promising results in using AI to pre

### 미들웨어

미들웨어는 실행의 다양한 단계에서 에이전트 동작을 커스터마이징하기 위한 강력한 확장성을 제공합니다.

**미들웨어 사용 사례:**

- 모델 호출 전 상태 처리 (예: 메시지 트리밍, 컨텍스트 주입)
- 모델 응답 수정 또는 검증 (예: 가드레일, 콘텐츠 필터링)
- 커스텀 로직으로 도구 실행 오류 처리
- 상태 또는 컨텍스트에 따른 동적 모델 선택
- 커스텀 로깅, 모니터링 또는 분석 추가